
#  Optimizing Pickup Data in Last Mile Logistics
## Data Preprocessing 
---


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('merged_pickup_data.csv')
df.head(10)

,order_id,region_id,city,courier_id,accept_time,time_window_start,time_window_end,lng,lat,aoi_id,aoi_type,pickup_time,pickup_gps_time,pickup_gps_lng,pickup_gps_lat,accept_gps_time,accept_gps_lng,accept_gps_lat,ds
0,483671,3,Chongqing,1518,08-14 07:57:00,08-14 09:00:00,08-14 11:00:00,106.46877,29.47204,218,14,08-14 09:38:00,NaN,NaN,NaN,NaN,NaN,NaN,814
1,1746131,3,Chongqing,4706,10-09 07:46:00,10-09 09:00:00,10-09 11:00:00,106.46872,29.47200,218,14,10-09 09:42:00,NaN,NaN,NaN,NaN,NaN,NaN,1009
2,2301722,3,Chongqing,4706,10-09 13:57:00,10-09 13:57:00,10-09 15:57:00,106.46869,29.47191,218,14,10-09 15:53:00,10-09 15:53:00,106.46821,29.46771,10-09 13:56:00,106.46929,29.47231,1009
3,3788723,3,Chongqing,4706,05-19 08:13:00,05-19 11:00:00,05-19 13:00:00,106.46878,29.47208,218,14,05-19 11:59:00,NaN,NaN,NaN,NaN,NaN,NaN,519
4,713435,3,Chongqing,4706,05-22 08:16:00,05-22 09:00:00,05-22 11:00:00,106.46813,29.47228,218,14,05-22 10:40:00,05-22 10:40:00,106.46827,29.47270,NaN,NaN,NaN,522
5,2718201,3,Chongqing,4706,05-19 07:43:00,05-19 09:00:00,05-19 11:00:00,106.46869,29.47206,218,14,05-19 09:56:00,05-19 09:55:00,106.46806,29.47277,NaN,NaN,NaN,519
6,1814278,3,Chongqing,4706,10-09 10:36:00,10-09 17:00:00,10-09 19:00:00,106.46867,29.47200,218,14,10-09 18:44:00,10-09 18:44:00,106.46984,29.42807,NaN,NaN,NaN,1009
7,2308168,3,Chongqing,4706,05-17 09:17:00,05-17 13:00:00,05-17 15:00:00,106.46875,29.47199,218,14,05-17 11:36:00,05-17 11:36:00,106.47008,29.47085,NaN,NaN,NaN,517
8,4951583,3,Chongqing,4706,05-19 07:39:00,05-19 09:00:00,05-19 11:00:00,106.46812,29.47229,218,14,05-19 09:48:00,05-19 09:48:00,106.46814,29.47271,NaN,NaN,NaN,519
9,3211213,3,Chongqing,4706,05-21 13:18:00,05-22 11:00:00,05-22 13:00:00,106.46872,29.47197,218,14,05-22 11:26:00,05-22 11:25:00,106.47038,29.47111,NaN,NaN,NaN,522


---






### Understanding the dataset - Statistical overview

In [3]:
df.columns

Index(['order_id', 'region_id', 'city', 'courier_id', 'accept_time',
       'time_window_start', 'time_window_end', 'lng', 'lat', 'aoi_id',
       'aoi_type', 'pickup_time', 'pickup_gps_time', 'pickup_gps_lng',
       'pickup_gps_lat', 'accept_gps_time', 'accept_gps_lng', 'accept_gps_lat',
       'ds'],
      dtype='object')

In [4]:
df.shape

(6136147, 19)

In [5]:
df['city'].unique()

array(['Chongqing', 'Hangzhou', 'Jilin', 'Shanghai', 'Yantai'],
      dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6136147 entries, 0 to 6136146
Data columns (total 19 columns):
 #   Column             Dtype  
---  ------             -----  
 0   order_id           int64  
 1   region_id          int64  
 2   city               object 
 3   courier_id         int64  
 4   accept_time        object 
 5   time_window_start  object 
 6   time_window_end    object 
 7   lng                float64
 8   lat                float64
 9   aoi_id             int64  
 10  aoi_type           int64  
 11  pickup_time        object 
 12  pickup_gps_time    object 
 13  pickup_gps_lng     float64
 14  pickup_gps_lat     float64
 15  accept_gps_time    object 
 16  accept_gps_lng     float64
 17  accept_gps_lat     float64
 18  ds                 int64  
dtypes: float64(6), int64(6), object(7)
memory usage: 889.5+ MB


In [7]:
df[['accept_time', 'pickup_time']].sample(5)

,accept_time,pickup_time
1301236,07-28 09:07:00,07-28 14:49:00
70004,05-12 07:56:00,05-12 09:57:00
1409937,10-28 08:19:00,10-28 09:56:00
4152551,07-17 08:05:00,07-17 09:31:00
1138680,10-12 08:46:00,10-12 10:05:00


In [8]:
df.describe()

,order_id,region_id,courier_id,lng,lat,aoi_id,aoi_type,pickup_gps_lng,pickup_gps_lat,accept_gps_lng,accept_gps_lat,ds
count,6.136147e+06,6.136147e+06,6.136147e+06,6.136147e+06,6.136147e+06,6.136147e+06,6.136147e+06,4.343285e+06,4.343285e+06,3.353776e+06,3.353776e+06,6.136147e+06
mean,3.090931e+06,6.452570e+01,7.889059e+03,1.183564e+02,3.230873e+01,1.215777e+04,5.043091e+00,1.183813e+02,3.257753e+01,1.185414e+02,3.270376e+01,7.803486e+02
std,1.775744e+06,3.791702e+01,4.565536e+03,5.720413e+00,3.680536e+00,7.131654e+03,5.848862e+00,5.792829e+00,3.865521e+00,5.705898e+00,3.930465e+00,1.661045e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,1.053727e+02,2.810210e+01,0.000000e+00,0.000000e+00,1.025392e+02,1.999148e+01,1.025391e+02,2.000799e+01,5.010000e+02
25%,1.559888e+06,3.300000e+01,4.046000e+03,1.199455e+02,3.019368e+01,5.921000e+03,1.000000e+00,1.199407e+02,3.020180e+01,1.199479e+02,3.021873e+01,6.220000e+02
50%,3.093924e+06,6.600000e+01,7.878000e+03,1.202768e+02,3.043618e+01,1.217000e+04,1.000000e+00,1.202858e+02,3.094079e+01,1.203087e+02,3.101978e+01,8.040000e+02
75%,4.627960e+06,9.500000e+01,1.175800e+04,1.213778e+02,3.135063e+01,1.820100e+04,1.400000e+01,1.214115e+02,3.695500e+01,1.214257e+02,3.718493e+01,9.200000e+02
max,6.161997e+06,1.340000e+02,1.606600e+04,1.303666e+02,4.442747e+01,2.454300e+04,1.400000e+01,1.303508e+02,4.567432e+01,1.276731e+02,4.567428e+01,1.031000e+03


In [9]:
#checking counts
df.count().to_frame(name='Count_Rows')

,Count_Rows
order_id,6136147
region_id,6136147
city,6136147
courier_id,6136147
accept_time,6136147
time_window_start,6136147
time_window_end,6136147
lng,6136147
lat,6136147
aoi_id,6136147


---

## Data Cleaning
##### Handling Missing Values
##### Checking Duplicates
##### Verifing Data Types

In [10]:
#checking missing values
df.isnull().sum()

order_id                   0
region_id                  0
city                       0
courier_id                 0
accept_time                0
time_window_start          0
time_window_end            0
lng                        0
lat                        0
aoi_id                     0
aoi_type                   0
pickup_time                0
pickup_gps_time      1792862
pickup_gps_lng       1792862
pickup_gps_lat       1792862
accept_gps_time      2782371
accept_gps_lng       2782371
accept_gps_lat       2782371
ds                         0
dtype: int64

In [11]:
# Checking for Missing Values in Essential Columns from gitbook
df[['accept_time', 'pickup_time', 'lng', 'lat', 'courier_id']].isnull().sum().to_frame(name='Essential Columns Missing Values')

,Essential Columns Missing Values
accept_time,0
pickup_time,0
lng,0
lat,0
courier_id,0


In [12]:
# Impute missing 'accept_gps_lng' and 'accept_gps_lat' values with the mean of their corresponding city groups
df['accept_gps_lng'] = df.groupby('city')['accept_gps_lng'].transform(lambda x: x.fillna(x.mean()))
df['accept_gps_lat'] = df.groupby('city')['accept_gps_lat'].transform(lambda x: x.fillna(x.mean()))

In [13]:
df['pickup_gps_lng'] = df.groupby('city')['pickup_gps_lng'].transform(lambda x: x.fillna(x.mean()))
df['pickup_gps_lat'] = df.groupby('city')['pickup_gps_lat'].transform(lambda x: x.fillna(x.mean()))

In [14]:
#verifying missing
df.isnull().sum()


order_id                   0
region_id                  0
city                       0
courier_id                 0
accept_time                0
time_window_start          0
time_window_end            0
lng                        0
lat                        0
aoi_id                     0
aoi_type                   0
pickup_time                0
pickup_gps_time      1792862
pickup_gps_lng             0
pickup_gps_lat             0
accept_gps_time      2782371
accept_gps_lng             0
accept_gps_lat             0
ds                         0
dtype: int64

<span style="color:DarkRed">

- Dataset contains ~2M null values in 'pickup_gps_time' and 'accept_gps_time' columns. 


- These nulls are kept since removing them would drop 1/3 of our data (6.1M -> 4.1M rows) and these columns 
 aren't needed for our ML model features anyway.



</span>


In [15]:
#checking dup
df[df.duplicated()]

,order_id,region_id,city,courier_id,accept_time,time_window_start,time_window_end,lng,lat,aoi_id,aoi_type,pickup_time,pickup_gps_time,pickup_gps_lng,pickup_gps_lat,accept_gps_time,accept_gps_lng,accept_gps_lat,ds


In [16]:
df.dtypes

order_id               int64
region_id              int64
city                  object
courier_id             int64
accept_time           object
time_window_start     object
time_window_end       object
lng                  float64
lat                  float64
aoi_id                 int64
aoi_type               int64
pickup_time           object
pickup_gps_time       object
pickup_gps_lng       float64
pickup_gps_lat       float64
accept_gps_time       object
accept_gps_lng       float64
accept_gps_lat       float64
ds                     int64
dtype: object

In [17]:
# converting dtypes
df['accept_time'] = pd.to_datetime('2024-' + df['accept_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['accept_gps_time'] = pd.to_datetime('2024-' + df['accept_gps_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['pickup_time'] = pd.to_datetime('2024-' + df['pickup_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['pickup_gps_time'] = pd.to_datetime('2024-' + df['pickup_gps_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lng'] = pd.to_numeric(df['lng'], errors='coerce')

df['aoi_type'] = df['aoi_type'].astype('category')

In [18]:
df['time_window_start'] = pd.to_datetime('2024-' + df['time_window_start'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['time_window_end'] = pd.to_datetime('2024-' + df['time_window_end'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [19]:
df.dtypes

order_id                      int64
region_id                     int64
city                         object
courier_id                    int64
accept_time          datetime64[ns]
time_window_start    datetime64[ns]
time_window_end      datetime64[ns]
lng                         float64
lat                         float64
aoi_id                        int64
aoi_type                   category
pickup_time          datetime64[ns]
pickup_gps_time      datetime64[ns]
pickup_gps_lng              float64
pickup_gps_lat              float64
accept_gps_time      datetime64[ns]
accept_gps_lng              float64
accept_gps_lat              float64
ds                            int64
dtype: object

In [20]:
df.sample(5)

,order_id,region_id,city,courier_id,accept_time,time_window_start,time_window_end,lng,lat,aoi_id,aoi_type,pickup_time,pickup_gps_time,pickup_gps_lng,pickup_gps_lat,accept_gps_time,accept_gps_lng,accept_gps_lat,ds
1395048,745532,4,Hangzhou,13847,2024-06-12 12:57:00,2024-06-12 13:00:00,2024-06-12 15:00:00,120.25469,30.13096,6213,14,2024-06-12 14:36:00,2024-06-12 14:36:00,120.255530,30.132570,2024-06-12 12:57:00,120.254740,30.131430,612
2733216,1878246,80,Hangzhou,6484,2024-10-02 13:40:00,2024-10-02 15:00:00,2024-10-02 17:00:00,119.93807,30.03247,18345,1,2024-10-02 15:22:00,NaT,120.111544,30.257508,NaT,120.112550,30.257184,1002
3260215,2903573,100,Hangzhou,8691,2024-06-26 08:13:00,2024-06-26 09:00:00,2024-06-26 11:00:00,120.24811,30.37786,14763,14,2024-06-26 10:06:00,2024-06-26 10:05:00,120.246850,30.378700,2024-06-26 08:11:00,120.256130,30.388310,626
2810039,4488975,82,Hangzhou,7425,2024-06-21 14:57:00,2024-06-21 15:00:00,2024-06-21 17:00:00,120.18163,30.26601,17126,1,2024-06-21 15:32:00,NaT,120.111544,30.257508,NaT,120.112550,30.257184,621
111855,6031315,22,Chongqing,13040,2024-09-29 07:14:00,2024-09-29 09:00:00,2024-09-29 11:00:00,106.49615,29.53346,17433,4,2024-09-29 08:49:00,NaT,107.013589,29.828223,NaT,107.031439,29.842615,929


---

## Derived Features:
#### - Calculating NEW columns: `Time Window Duration` And `Task Duration` AND `Distance Between Acceptance to Pickup` columns

In [21]:
#adding time_window_duration column
df['time_window_duration'] = df['time_window_end'] - df['time_window_start']

In [22]:
df['time_window_duration'].describe()

count                      6136147
mean     0 days 02:07:56.585068773
std      0 days 01:31:09.687214997
min                0 days 00:04:00
25%                0 days 02:00:00
50%                0 days 02:00:00
75%                0 days 02:00:00
max                1 days 06:59:00
Name: time_window_duration, dtype: object

In [23]:
df[['time_window_start', 'time_window_end', 'time_window_duration']].sample(10)

,time_window_start,time_window_end,time_window_duration
4812935,2024-10-15 09:00:00,2024-10-15 11:00:00,0 days 02:00:00
5861716,2024-08-13 11:00:00,2024-08-13 13:00:00,0 days 02:00:00
3777400,2024-07-04 09:00:00,2024-07-04 11:00:00,0 days 02:00:00
430495,2024-06-07 15:00:00,2024-06-07 17:00:00,0 days 02:00:00
4658201,2024-09-29 11:00:00,2024-09-29 13:00:00,0 days 02:00:00
3835623,2024-10-16 09:00:00,2024-10-16 11:00:00,0 days 02:00:00
4747798,2024-10-07 11:00:00,2024-10-07 13:00:00,0 days 02:00:00
303277,2024-06-11 14:24:00,2024-06-11 16:24:00,0 days 02:00:00
2728373,2024-10-03 10:40:00,2024-10-03 12:40:00,0 days 02:00:00
3364222,2024-09-12 09:00:00,2024-09-12 11:00:00,0 days 02:00:00


In [24]:
#adding task duration column
df['task_duration'] = df['pickup_time'] - df['accept_time']

df[['accept_time', 'pickup_time', 'task_duration']].head()

,accept_time,pickup_time,task_duration
0,2024-08-14 07:57:00,2024-08-14 09:38:00,0 days 01:41:00
1,2024-10-09 07:46:00,2024-10-09 09:42:00,0 days 01:56:00
2,2024-10-09 13:57:00,2024-10-09 15:53:00,0 days 01:56:00
3,2024-05-19 08:13:00,2024-05-19 11:59:00,0 days 03:46:00
4,2024-05-22 08:16:00,2024-05-22 10:40:00,0 days 02:24:00


In [25]:
df['task_duration'].describe()

count                      6136147
mean     0 days 03:41:34.258113438
std      0 days 06:58:21.749930310
min                0 days 00:00:00
25%                0 days 01:04:00
50%                0 days 01:58:00
75%                0 days 03:17:00
max               38 days 02:19:00
Name: task_duration, dtype: object

In [26]:
#adding distance column using the Euclidean formula
df['distance'] = np.sqrt((df['pickup_gps_lng'] - df['accept_gps_lng'])**2 +
                         (df['pickup_gps_lat'] - df['accept_gps_lat'])**2)

df[['pickup_gps_lng', 'accept_gps_lng', 'pickup_gps_lat', 'accept_gps_lat', 'distance']].sample(10)

,pickup_gps_lng,accept_gps_lng,pickup_gps_lat,accept_gps_lat,distance
5589146,121.136760,121.122470,37.574330,37.583500,0.016979
5656878,120.804760,121.044323,37.802940,37.451944,0.424957
3586670,121.699450,121.700260,31.182790,31.182840,0.000812
5878510,121.037815,121.044323,37.444311,37.451944,0.010031
1066399,108.040050,108.039920,30.302870,30.292000,0.010871
4018254,121.485100,121.489600,31.305590,31.301610,0.006008
1750652,119.722930,119.692750,30.228840,30.218130,0.032024
5684098,121.157680,121.044323,36.777030,37.451944,0.684367
122593,107.013589,107.031439,29.828223,29.842615,0.022929
3845563,121.493143,121.210690,31.233340,31.017070,0.355742


In [27]:
df['distance'].describe()

count    6.136147e+06
mean     1.042813e-01
std      2.745168e-01
min      0.000000e+00
25%      1.905151e-03
50%      9.222001e-03
75%      5.383025e-02
max      1.815873e+01
Name: distance, dtype: float64

---

## Outlier Detection

- #### ```task_duration``` : Identifing abnormally long or short delivery durations.

In [28]:
# Calculating Q1 and Q3
Q1 = df['task_duration'].quantile(0.25)
Q3 = df['task_duration'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


lower_bound, upper_bound

(Timedelta('-1 days +21:44:30'), Timedelta('0 days 06:36:30'))

In [29]:
# Filtering out outliers
df = df[df['task_duration'].between(lower_bound, upper_bound)]
df.sample(5)

,order_id,region_id,city,courier_id,accept_time,time_window_start,time_window_end,lng,lat,aoi_id,...,pickup_gps_time,pickup_gps_lng,pickup_gps_lat,accept_gps_time,accept_gps_lng,accept_gps_lat,ds,time_window_duration,task_duration,distance
5617480,5715847,117,Yantai,5009,2024-07-28 07:35:00,2024-07-28 09:00:00,2024-07-28 11:00:00,121.44181,37.46060,11664,...,2024-07-28 09:38:00,121.443020,37.460890,NaT,121.044323,37.451944,728,0 days 02:00:00,0 days 02:03:00,0.398797
2542785,3234311,77,Hangzhou,13097,2024-08-26 11:48:00,2024-08-26 13:00:00,2024-08-26 15:00:00,120.17583,30.27737,2808,...,2024-08-26 12:01:00,120.176080,30.277310,2024-08-26 11:44:00,120.174880,30.275760,826,0 days 02:00:00,0 days 00:13:00,0.001960
903260,5912491,88,Chongqing,3471,2024-07-16 14:19:00,2024-07-16 15:00:00,2024-07-16 17:00:00,106.48166,29.58285,11487,...,NaT,107.013589,29.828223,NaT,107.031439,29.842615,716,0 days 02:00:00,0 days 02:06:00,0.022929
3302058,3112655,100,Hangzhou,13232,2024-06-07 11:51:00,2024-06-07 09:00:00,2024-06-07 23:59:00,120.25040,30.39019,23194,...,NaT,120.111544,30.257508,NaT,120.112550,30.257184,607,0 days 14:59:00,0 days 01:45:00,0.001057
5528854,5150785,113,Yantai,3126,2024-10-14 14:50:00,2024-10-14 15:00:00,2024-10-14 17:00:00,121.25569,37.51145,21776,...,NaT,121.037815,37.444311,NaT,121.044323,37.451944,1014,0 days 02:00:00,0 days 00:11:00,0.010031


In [30]:
df['task_duration'].describe()

count                      5460748
mean     0 days 02:00:41.967876928
std      0 days 01:23:59.708834992
min                0 days 00:00:00
25%                0 days 00:58:00
50%                0 days 01:46:00
75%                0 days 02:42:00
max                0 days 06:36:00
Name: task_duration, dtype: object

---

## Overview & Saving Cleaned Data

In [31]:
df.columns

Index(['order_id', 'region_id', 'city', 'courier_id', 'accept_time',
       'time_window_start', 'time_window_end', 'lng', 'lat', 'aoi_id',
       'aoi_type', 'pickup_time', 'pickup_gps_time', 'pickup_gps_lng',
       'pickup_gps_lat', 'accept_gps_time', 'accept_gps_lng', 'accept_gps_lat',
       'ds', 'time_window_duration', 'task_duration', 'distance'],
      dtype='object')

In [32]:
df.shape

(5460748, 22)

In [33]:
df.iloc[:5, :12]


,order_id,region_id,city,courier_id,accept_time,time_window_start,time_window_end,lng,lat,aoi_id,aoi_type,pickup_time
0,483671,3,Chongqing,1518,2024-08-14 07:57:00,2024-08-14 09:00:00,2024-08-14 11:00:00,106.46877,29.47204,218,14,2024-08-14 09:38:00
1,1746131,3,Chongqing,4706,2024-10-09 07:46:00,2024-10-09 09:00:00,2024-10-09 11:00:00,106.46872,29.47200,218,14,2024-10-09 09:42:00
2,2301722,3,Chongqing,4706,2024-10-09 13:57:00,2024-10-09 13:57:00,2024-10-09 15:57:00,106.46869,29.47191,218,14,2024-10-09 15:53:00
3,3788723,3,Chongqing,4706,2024-05-19 08:13:00,2024-05-19 11:00:00,2024-05-19 13:00:00,106.46878,29.47208,218,14,2024-05-19 11:59:00
4,713435,3,Chongqing,4706,2024-05-22 08:16:00,2024-05-22 09:00:00,2024-05-22 11:00:00,106.46813,29.47228,218,14,2024-05-22 10:40:00


In [34]:
df.iloc[:5,12:]

,pickup_gps_time,pickup_gps_lng,pickup_gps_lat,accept_gps_time,accept_gps_lng,accept_gps_lat,ds,time_window_duration,task_duration,distance
0,NaT,107.013589,29.828223,NaT,107.031439,29.842615,814,0 days 02:00:00,0 days 01:41:00,0.022929
1,NaT,107.013589,29.828223,NaT,107.031439,29.842615,1009,0 days 02:00:00,0 days 01:56:00,0.022929
2,2024-10-09 15:53:00,106.468210,29.467710,2024-10-09 13:56:00,106.469290,29.472310,1009,0 days 02:00:00,0 days 01:56:00,0.004725
3,NaT,107.013589,29.828223,NaT,107.031439,29.842615,519,0 days 02:00:00,0 days 03:46:00,0.022929
4,2024-05-22 10:40:00,106.468270,29.472700,NaT,107.031439,29.842615,522,0 days 02:00:00,0 days 02:24:00,0.673792


In [35]:
df.to_csv('cleaned_pickup_data.csv', index=False)